# Amazon Web Scraping using Selenium

## 1. Importing the necessary libraries

In [ ]:
import os
import re
import selenium
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from random import sample
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.common import exceptions
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
nltk.download('vader_lexicon')
selenium.__version__
print("done")

## 2. SET DIRECTORY

In [ ]:
os.chdir("")
cwd = os.getcwd()
print(cwd)

## 3. Begin the scraping

### Open Automated Webdriver

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install() , options=options)

### Go to Core Website/Link

In [ ]:
# Go to Core Website/Link Where you want to scrape the category and product links

driver.get('https://www.amazon.com/Best-Sellers-Electronics-Computers-Tablets/zgbs/electronics/13896617011/ref=zg_bs_nav_electronics_2_541966')

### Getting the Categories (Computers, Tablets, Desktop)

In [ ]:
# Getting Categories 

EL_sidebar = driver.find_elements(By.XPATH,f"//div[@role='group']")


if len(EL_sidebar)==1:
    EL_sidebar = EL_sidebar[0]
else:
    print('Tag Mismatch')

catlist = EL_sidebar.find_elements(By.XPATH,f".//a")
category_list = []
link_list = []
for i in catlist:
    category_list.append(i.text) 
    link_list.append(i.get_attribute('href'))
    
print(category_list)
print(link_list)

### Start Scraping Product Links (review_page)

In [ ]:
# Create Empty Array Variable
FINAL = []

# Start Scraping

for cat_pick,link_pick in tqdm(zip(category_list,link_list)):
    
    driver.get(link_pick)


    # Scroll To Load all the data 
    y = 0
    for timer in range(0,15):

        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 1500
        time.sleep(.3)
    time.sleep(2)
    # Scrape All The Items 
    items = driver.find_elements(By.XPATH,f"//div[@id='gridItemRoot']")
    for item in items:
        name = item.find_elements(By.XPATH,f".//div[@class='_cDEzb_p13n-sc-css-line-clamp-3_g3dy1']")
        if len(name) == 1:
            name = name[0].text

        rank = item.find_elements(By.XPATH,f".//span[@class='zg-bdg-text']")
        if len(rank) == 1:
            rank = rank[0].text
        
        price = item.find_elements(By.XPATH,f".//span[@class='a-size-base a-color-price']")
        if len(price) == 1:
            price = price[0].text

        offer = item.find_elements(By.XPATH,f".//span[@class='a-color-secondary']//span[@class='a-size-base']")
        if len(offer) == 1:
            offer = offer[0].text
            

        stareview = item.find_elements(By.XPATH,f".//div[@class='a-icon-row']//a[@class='a-link-normal']")
        if len(stareview) == 1:
            reviews = stareview[0].text
            stars = stareview[0].get_attribute('title')
            review_page = stareview[0].get_attribute('href')

        meta = {
            'name' : name,
            'rank' : rank,
            'reviews' : reviews,
            'stars' : stars,
            'review_page' : review_page,
            'cat' : cat_pick,
            'print' : price,
            'offer' : offer,
        }

        FINAL.append(meta)
print("Done")

### Check Product Links

In [ ]:
FINALdf = pd.DataFrame(FINAL)
FINALdf.to_csv("D:/desktop/3-2-23 Testing/links.csv")

### Scraping the reviews from the scraped product links 

In [ ]:
# STEP 1 (SCRAPING - Whole Computer and Tablets Category Links)

df = pd.read_csv("D:/desktop/3-2-23 Testing/links.csv")
df

In [ ]:
# STEP 2 (CHANGE AFTER EACH PRODUCT - CHOOSE SPECIFIC PRODUCT)

dfnew = df.loc[(df['rank'] == '#1') & (df['cat'] == 'Desktops')]
dfnew = dfnew.reset_index(drop=True)
dfnew = dfnew.iloc[:,1:]
dfnew

In [ ]:
# STEP 3 (START SCRAPING OF REVIEWSTAR AND REVIEWBODY
# CREATE EMPTY ARRAY VARIABLE

FINALFINAL = []

for i in tqdm(range(len(dfnew))):
    reviewpage = dfnew.loc[i]['review_page']
    name = dfnew.loc[i]['name']
    categoryRank = dfnew.loc[i]['rank']
    noRatings = dfnew.loc[i]['reviews']
    stars = dfnew.loc[i]['stars'].split(' ')[0]
    category = dfnew.loc[i]['cat']
    cost = dfnew.loc[i]['print']
    
    driver.get(reviewpage)
    
    product = driver.find_elements(By.XPATH,f"//a[@data-hook='product-link']")

    product_name = ''
    product_url = ''
    
    if len(product) == 1:
        product_name = product[0].text
        product_url = product[0].get_attribute('href')  

    number_sang_reviews = ''
    number_sang_reviews = driver.find_elements(By.XPATH,f"//div[@data-hook='cr-filter-info-review-rating-count']")
    number_sang_reviews = number_sang_reviews[0].text
    number_sang_reviews = number_sang_reviews.split("ratings, ")
    number_sang_reviews = number_sang_reviews[1]
    number_sang_reviews = re.sub(r'[^0-9]', '', number_sang_reviews)
    number_sang_reviews = int(number_sang_reviews)

    libot_count = number_sang_reviews//10
    print(libot_count)

    libot_count = 2
    
    for libot in range(libot_count):
        time.sleep(2)
        reviews = driver.find_elements(By.XPATH,f"//div[@data-hook='review']")
        nextpage = driver.find_element(By.CLASS_NAME, 'a-last')
        REVIEWBODY = []
        REVIEWSTAR = []
        REVIEWPAGEURL = []
        USERNAME = []
        VERIFICATION = []
        
        for review in reviews:
            review_body = ''
            review_star = ''
            review_page_url = ''
            review_username = ''
            review_verification = ''


            review_page_url = driver.current_url


            review_body_el = review.find_elements(By.XPATH,f".//span[@data-hook='review-body']")

            if len(review_body_el)== 1:
                review_body = review_body_el[0].text


            review_star_el = review.find_elements(By.XPATH,f".//i[@data-hook='review-star-rating']")

            if len(review_star_el)== 1:
                review_star = review_star_el[0].get_attribute('class')

         
            review_username_el = review.find_elements(By.XPATH,f".//span[@class='a-profile-name']")

            if len(review_username_el)== 1:
                review_username = review_username_el[0].text

            review_verification_el = review.find_elements(By.XPATH,f".//span[@data-hook='avp-badge']")

            if len(review_verification_el)== 1:
                review_verification = review_verification_el[0].text

            REVIEWBODY.append({
                'review_body':review_body
            })
            
            REVIEWSTAR.append({
                'review_star':review_star,
            })
            USERNAME.append({
                'review_username':review_username,
            })
            VERIFICATION.append({
                'review_verification':review_verification,
            })

            newRow = {
                'USERNAME':USERNAME,
                'VERIFICATION':VERIFICATION,
                'reviewpage':reviewpage,
                'name':name,
                'categoryRank':categoryRank,
                'category':category,
                'cost':cost,
                'REVIEWBODY':REVIEWBODY,
                'REVIEWSTAR':REVIEWSTAR,
                'product_url':product_url,
                'product_name':product_name,
                'review_page_url':review_page_url,
                'no_of_reviews':number_sang_reviews
            }

        FINALFINAL.append(newRow)


        FINALFINAL_DF = pd.DataFrame(FINALFINAL)


        FINALFINAL_DF.to_csv("D:/desktop/3-2-23 Testing/scraped-reviews.csv")
        time.sleep(2)
        nextpage.click()
    time.sleep(2)

In [ ]:
# Use the glob() function to list only the csv files from the working directory.
#list all csv files only

csv_files = glob.glob("C:/Users/DVF2/Desktop/Electronics 11-9-22/1pm" + "/*.csv")
csv_files

In [ ]:
# INITIALIZE DATAFRAME/CREATE EMPTY DATAFRAME
df_append = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file)
            df_append = df_append.append(df_temp, ignore_index=True)
sample_reviews = df_append